In [31]:
%load_ext autoreload
# from google.colab import drive
# drive.mount('/content/drive')
#%cd "/content/drive/MyDrive/ldh"
# import nltk
# nltk.download('punkt')

%cd "/Users/danielpham/Google Drive/ldh"
%pip install transformers datasets pytorch-lightning nltk matplotlib "ray[tune]"
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/danielpham/Google Drive/ldh
You should consider upgrading via the '/Users/danielpham/.pyenv/versions/3.8.7/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Populating the interactive namespace from numpy and matplotlib


## Data Preparation
- Data is loaded through LDHDataModule. 
- Returned 

In [32]:
%autoreload 2 
from src.LDHDataModule import LDHDataModule
from src.LightningReapp import LightningReapp, ReappTrainer

ldhdata = LDHDataModule(batch_size=16, strat='far', kfolds=3)

Loading cached processed dataset at /Users/danielpham/Google Drive/ldh/output/training/far/cache-f38e1c59d490f503.arrow
  1%|          | 282/32109 [00:00<00:11, 2814.65ex/s]Training data loaded from disk.
Encoding Train Data:
Evaluation data loaded from disk.
Encoding Test Data:
100%|██████████| 32109/32109 [00:12<00:00, 2590.52ex/s]


In [67]:
%autoreload 2 
from datetime import datetime
from pathlib import Path

import torch
import pandas as pd 
import pytorch_lightning as lit 

from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, Callback
from pl_bolts.callbacks import PrintTableMetricsCallback
from ray.tune.integration.pytorch_lightning import TuneReportCallback

save_dir = Path.cwd() / 'output'
logger = lit.loggers.TensorBoardLogger(save_dir=save_dir)

callback_tuner = TuneReportCallback({
    "loss": "val_loss",
    #"mean_accuracy": "val_accuracy"
    },
    on="validation_end")

# TODO: Refactor into main loop 
trainer = ReappTrainer(
    num_folds = 3,
    fast_dev_run=1,
    logger=logger,
    max_epochs=50,
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    terminate_on_nan=True,
    weights_summary="top",
    weights_save_path=save_dir,
    callbacks=[callback_earlystopping, callback_checkpoint]
  )


EarlyStopping mode set to min for monitoring val_loss.
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).


In [69]:
### TUNING HYPERPARAMETERS
from ray import tune
def train_tune(config, epochs=10, gpus=0):
    model = LightningReapp(config)
    trainer = ReappTrainer(
        num_folds=3,
        fast_dev_run=1,
        max_epochs=epochs,
        gpus=gpus,
        progress_bar_refresh_rate=30,
        callbacks=[callback_tuner])
    trainer.fit(model, ldhdata)

tune_config = {
    "lr": tune.loguniform(1e-4, 1e-1),
}
# https://medium.com/distributed-computing-with-ray/scaling-up-pytorch-lightning-hyperparameter-tuning-with-ray-tune-4bd9e1ff9929
from functools import partial
tune.run(
  partial(train_tune, gpus=0),
  config=tune_config,
  num_samples=2)

2021-02-16 00:32:21,383	WARNING experiment.py:291 -- No name detected on trainable. Using DEFAULT.
2021-02-16 00:32:21,384	INFO registry.py:64 -- Detected unknown callable for trainable. Converting to class.


Training: 0it [00:00, ?it/s]
Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] 
(pid=88311) GPU available: False, used: False
(pid=88311) TPU available: None, using: 0 TPU cores
(pid=88311) Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
(pid=88311) 
(pid=88311)   | Name       | Type             | Params
(pid=88311) ------------------------------------------------
(pid=88311) 0 | model      | DistilBertModel  | 66.4 M
(pid=88311) 1 | avg        | AvgPool1d        | 0     
(pid=88311) 2 | classifier | Sequential       | 629 K 
(pid=88311) 3 | loss       | MeanSquaredError | 0     
(pid=88311) ------------------------------------------------
(pid=88311) 67.0 M    Trainable params
(pid=88311) 0         Non-trainable params
(pid=88311) 67.0 M    Total params
(pid=88571) GPU available: False, used: False
(pid=88571) TPU available: None, using: 0 TPU cores
(pid=88571) Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(e

Epoch 0: 100%|██████████| 1/1 [00:10<00:00, 10.34s/it, loss=13.5]
Trial DEFAULT_b947c_00001 completed. Last result: 


2021-02-16 00:32:36,936	INFO tune.py:450 -- Total run time: 15.55 seconds (15.45 seconds for the tuning loop).


In [80]:
### K-FOLD CV
from datetime import datetime
config = {
    'lr': 1e-3
}
# Stops when the val loss stops changing
callback_earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=True)
# Saves the top 3 checkpoints
callback_checkpoint = ModelCheckpoint(monitor="val_loss", mode='min', save_top_k=3,
        #filename='reappraisal-model-frozenbert-{epoch_02d}-{val_loss:.2f}-gpu1',
        dirpath=save_dir)
for i in range(3):
    train_dl = ldhdata.get_train_dataloader(i)
    val_dl = ldhdata.get_val_dataloader(i)

    strat = ldhdata.strat
    dt = datetime.now() # get the datetime 
    split = i # Current split being trained
    model = LightningReapp(config)
    trainer = ReappTrainer(
        num_folds = 3,
        fast_dev_run=1,
        logger=logger,
        max_epochs=50,
        gradient_clip_val=1.0,
        progress_bar_refresh_rate=30,
        terminate_on_nan=True,
        weights_summary="top",
        weights_save_path=save_dir,
        callbacks=[callback_earlystopping])
    print(f"Training on split {i + 1}")
    trainer.fit(model, train_dl, val_dl)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).

  | Name       | Type             | Params
------------------------------------------------
0 | model      | DistilBertModel  | 66.4 M
1 | avg        | AvgPool1d        | 0     
2 | classifier | Sequential       | 629 K 
3 | loss       | MeanSquaredError | 0     
------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
Epoch 0: 100%|██████████| 2/2 [00:08<00:00,  4.26s/it]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 2/2 [00:10<00:00,  5.48s/it, loss=15.2]
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).

  | Name       | Type             | Params
------------------------------------------------
0 | model      | DistilBertModel

([<torch.utils.data.dataloader.DataLoader at 0x13ed72d00>,
  <torch.utils.data.dataloader.DataLoader at 0x13ecd46d0>])